In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *

# Py4JJavaError: An error occurred while calling o73.save.
# : java.lang.NoClassDefFoundError: org/apache/commons/httpclient/protocol/ProtocolSocketFactory
spark = (
    SparkSession
    .builder
    .master('spark://host.docker.internal:7077')
    .appName("pytest")
    # .config("spark.jars.packages", "org.elasticsearch:elasticsearch-hadoop:8.12.1")
    # .config("spark.jars.packages", "org.elasticsearch:elasticsearch-spark-30_2.12:7.12.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.0")
    .config("spark.jars.packages", "org.elasticsearch:elasticsearch-spark-30_2.12:7.12.0,commons-httpclient:commons-httpclient:3.1")
    # .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    # .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .getOrCreate()
)

:: loading settings :: url = jar:file:/opt/bitnami/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.elasticsearch#elasticsearch-spark-30_2.12 added as a dependency
commons-httpclient#commons-httpclient added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-02f245b2-ea74-42ec-9b57-912d9a6a0d33;1.0
	confs: [default]
	found org.elasticsearch#elasticsearch-spark-30_2.12;7.12.0 in central
	found org.scala-lang#scala-reflect;2.12.8 in central
	found org.slf4j#slf4j-api;1.7.6 in central
	found commons-logging#commons-logging;1.1.1 in central
	found javax.xml.bind#jaxb-api;2.3.1 in central
	found com.google.protobuf#protobuf-java;2.5.0 in central
	found org.apache.spark#spark-yarn_2.12;3.0.1 in central
	found commons-httpclient#commons-httpclient;3.1 in central
	found commons-codec#commons-codec;1.2 in central
:: resolution report :: resolve 403ms :: artifacts dl 15ms
	:: modules in use:
	com.google.protobuf#protobuf-java;2.5.0 from central in [default]
	comm

24/11/14 05:34:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [25]:
data = [
    ("Alice", 31, "New York"),
    ("Bob", 25, "Los Angeles"),
    ("Charlie", 35, "Chicago"),
]

df = spark.createDataFrame(data, ["name", "age", "city"])
df.show()

[Stage 40:===============================================>        (21 + 2) / 25]

+-------+---+-----------+
|   name|age|       city|
+-------+---+-----------+
|  Alice| 31|   New York|
|    Bob| 25|Los Angeles|
|Charlie| 35|    Chicago|
+-------+---+-----------+



In [33]:
data = [
    (1, "Alice", 32, "New York", 'x'),
    (2, "Bob", 25, "Los Angeles", 'y'),
    (3, "Charlie", 35, "Chicago", 'x'),
]

df = spark.createDataFrame(data, ["id", "name", "age", "city", "type"])
df.show()

+---+-------+---+-----------+----+
| id|   name|age|       city|type|
+---+-------+---+-----------+----+
|  1|  Alice| 32|   New York|   x|
|  2|    Bob| 25|Los Angeles|   y|
|  3|Charlie| 35|    Chicago|   x|
+---+-------+---+-----------+----+



In [7]:
!pip install elasticsearch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 543.1/543.1 kB 3.4 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 9.6 MB/s eta 0:00:00


In [35]:
es_nodes = "http://host.docker.internal:9200"
es_resource = "test_index/_doc"

# (
#     df.write 
#     .format("org.elasticsearch.spark.sql")
#     .option("es.net.ssl", "false")
#     .option("es.nodes.wan.only","true")
#     .option("es.nodes", es_nodes)  
#     .option("es.resource", es_resource)
#    .save("test_index/_doc")
# )

esconf={
    "es.mapping.id": "id",
    "es.mapping.exclude": "id",
    "es.nodes": "http://host.docker.internal",
    "es.port": "9200",
    "es.update.script.inline": "ctx._source.location = params.type",
    "es.write.operation": "upsert"
    
}
df.write.format("org.elasticsearch.spark.sql").options(**esconf).mode("append").save(es_resource)


# esconf["es.mapping.id"] = "_id"
# esconf["es.nodes"] = "localhost"
# esconf["es.port"] = "9200"
# esconf["es.update.script.inline"] = "ctx._source.location = params.location"
# esconf["es.update.script.params"] = "location:"
# esconf["es.write.operation"] = "upsert"

# df.write.format("org.elasticsearch.spark.sql")\
#     .option("es.nodes", es_nodes)\
#     .option("es.nodes.discovery", "false")\
#     .option("es.nodes.wan.only", "true")\
#     .option("es.index.auto.create", "true")\
#     .option("es.mapping.id", "id")\
#     .option("es.mapping.exclude", "id")\
#     # .option("es.write.operation", "upsert")\
#     .save("test_index/_doc")